#### Illustration of convolution of both discrete and continuous Pb loss distributions with a Gaussian distribution
##### Accompanyment to "Modeling apparent Pb loss in zircon U-Pb geochronology", submitted to Geochronology
By: Glenn R. Sharman, Department of Geosciences, University of Arkansas

In [ ]:
# Import required modules
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from scipy.stats import expon
from scipy.stats import rv_discrete
from scipy.stats import uniform
from scipy.stats import pareto
from scipy.stats import gamma
from scipy.stats import weibull_min
from scipy.stats import rayleigh
from scipy.stats import lognorm
from scipy.stats import halfnorm
from scipy.signal import convolve
import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'retina' # For improving matplotlib figure resolution
matplotlib.rcParams['pdf.fonttype'] = 42 # For allowing preservation of fonts upon importing into Adobe Illustrator
matplotlib.rcParams['ps.fonttype'] = 42

In [ ]:
# Mean and s.d. of normal distribution
mu = 0 # Mean of normal distribution, in %
omega = 1.5 # Standard deviation of the normal distribution, in %

xdif = 0.01 # X-axis discretization interval, in %
x1 = -100 # Lower x-axis for plotting, in %
x2 = 100 # Upper x-axis for plotting, in %
x = np.arange(x1, x2+xdif, xdif)

In [ ]:
# Define the Gaussian distribution pdf
rv_norm = norm(loc = mu, scale = omega)
norm_pdf = rv_norm.pdf(x)
norm_pdf = norm_pdf/np.sum(norm_pdf) # Normalize so area under the curve = 1

In [ ]:
# Define the parameters for the discrete distributions
xk_none = [0] # X-axis values in discrete distribution (i.e., Pb-loss, as pct)
pk_none = [1] # Y-axis values in discrete distribution (i.e., probability of Pb-loss, sum-to-one)

xk_const = [-3] # X-axis values in discrete distribution (i.e., Pb-loss, as pct)
pk_const = [1] # Y-axis values in discrete distribution (i.e., probability of Pb-loss, sum-to-one)

xk_mix = [0,-6] # X-axis values in discrete distribution (i.e., Pb-loss, as pct)
pk_mix = [0.80, 0.20] # Y-axis values in discrete distribution (i.e., probability of Pb-loss, sum-to-one)

# Define the discrete distributions
pb_loss_func_none = rv_discrete(values=(xk_none,pk_none))
pb_loss_func_const = rv_discrete(values=(xk_const,pk_const))
pb_loss_func_mix = rv_discrete(values=(xk_mix,pk_mix))
                                
# Get the cumulative probabilities for the values in x
# This is needed b/c the .pmf() function requires integers
cumulative_probs_none = pb_loss_func_none.cdf(x)
cumulative_probs_const = pb_loss_func_const.cdf(x)
cumulative_probs_mix = pb_loss_func_mix.cdf(x)
                                
# Compute the derivative of the cumulative probabilities so we can get back to pdf space
pdf_none = np.gradient(cumulative_probs_none, xdif)
pdf_const = np.gradient(cumulative_probs_const, xdif)
pdf_mix = np.gradient(cumulative_probs_mix, xdif)
                                
# Normalize so probabilities sum-to-one
pdf_none = pdf_none/np.sum(pdf_none)
pdf_const = pdf_const/np.sum(pdf_const)
pdf_mix = pdf_mix/np.sum(pdf_mix)

# Do the convolution
conv_none = convolve(pdf_none, norm_pdf, mode='same')
conv_const = convolve(pdf_const, norm_pdf, mode='same')
conv_mix = convolve(pdf_mix, norm_pdf, mode='same')

In [ ]:
# Make the figure
x_1 = -15
x_2 = 10

fig, axs = plt.subplots(4, 3, figsize=(10, 10))

axs[0,0].plot(xk_none, pb_loss_func_none.pmf(xk_none), 'ko', ms=12, mec='k')
axs[0,0].vlines(xk_none, 0, pb_loss_func_none.pmf(xk_none), colors='k', lw=4)
axs[0,0].axhline(1.0, ls='--', color='black')
axs[0,0].set_ylim(0,)
axs[0,0].set_xlim(-6.5, 0.5)
axs[0,0].set_ylabel('Relative probability')
axs[0,0].set_title('No Pb-loss')

axs[1,0].plot(x, cumulative_probs_none, '-', color='k')
axs[1,0].axhline(1.0, ls='--', color='black')
axs[1,0].set_ylim(0,)
axs[1,0].set_xlim(-6.5, 0.5)
axs[1,0].set_ylabel('Cumulative probability')

axs[2,0].plot(x, norm_pdf, '-', color='black', label='Original')
axs[2,0].plot(x, conv_none, '--', color='red', label='Pb-loss')
axs[2,0].set_xlim(x_1, x_2)
axs[2,0].set_ylim(0,)
axs[2,0].set_ylabel('Relative probability')
axs[2,0].legend()

axs[3,0].plot(x, np.cumsum(norm_pdf), '-', color='black', label='Original')
axs[3,0].plot(x, np.cumsum(conv_none), '--', color='red', label='Pb-loss')
axs[3,0].set_xlim(x_1, x_2)
axs[3,0].set_ylim(0,1)
axs[3,0].set_xlabel('Age offset (%)')
axs[3,0].set_ylabel('Cumulative probability')
axs[3,0].legend()

axs[0,1].plot(xk_const, pb_loss_func_const.pmf(xk_const), 'ko', ms=12, mec='k')
axs[0,1].vlines(xk_const, 0, pb_loss_func_const.pmf(xk_const), colors='k', lw=4)
axs[0,1].axhline(1.0, ls='--', color='black')
axs[0,1].set_ylim(0,)
axs[0,1].set_xlim(-6.5, 0.5)
axs[0,1].set_title('Constant Pb-loss')

axs[1,1].plot(x, cumulative_probs_const, '-', color='k')
axs[1,1].axhline(1.0, ls='--', color='black')
axs[1,1].set_ylim(0,)
axs[1,1].set_xlim(-6.5, 0.5)
axs[1,1].set_ylabel('Cumulative probability')

axs[2,1].plot(x, norm_pdf, '-', color='black', label='Original')
axs[2,1].plot(x, conv_const, '--', color='red', label='Pb-loss')
axs[2,1].set_xlim(x_1, x_2)
axs[2,1].set_ylim(0,)
axs[2,1].legend()

axs[3,1].plot(x, np.cumsum(norm_pdf), '-', color='black', label='Original')
axs[3,1].plot(x, np.cumsum(conv_const), '--', color='red', label='Pb-loss')
axs[3,1].set_xlim(x_1, x_2)
axs[3,1].set_ylim(0,1)
axs[3,1].set_xlabel('Age offset (%)')
axs[3,1].legend()

axs[0,2].plot(xk_mix, pb_loss_func_mix.pmf(xk_mix), 'ko', ms=12, mec='k')
axs[0,2].vlines(xk_mix, 0, pb_loss_func_mix.pmf(xk_mix), colors='k', lw=4)
axs[0,2].axhline(1.0, ls='--', color='black')
axs[0,2].set_ylim(0,)
axs[0,2].set_xlim(-6.5, 0.5)
axs[0,2].set_title('Isolated Pb-loss')

axs[1,2].plot(x, cumulative_probs_mix, '-', color='k')
axs[1,2].axhline(1.0, ls='--', color='black')
axs[1,2].set_ylim(0,)
axs[1,2].set_xlim(-6.5, 0.5)
axs[1,2].set_ylabel('Cumulative probability')

axs[2,2].plot(x, norm_pdf, '-', color='black', label='Original')
axs[2,2].plot(x, conv_mix, '--', color='red', label='Pb-loss')
axs[2,2].set_xlim(x_1, x_2)
axs[2,2].set_ylim(0,)
axs[2,2].legend()

axs[3,2].plot(x, np.cumsum(norm_pdf), '-', color='black', label='Original')
axs[3,2].plot(x, np.cumsum(conv_mix), '--', color='red', label='Pb-loss')
axs[3,2].set_xlim(x_1, x_2)
axs[3,2].set_ylim(0,1)
axs[3,2].set_xlabel('Age offset (%)')
axs[3,2].legend()

plt.tight_layout()

#### Next, model continuous distributions of Pb loss via convolution with a Gaussian distribution

In [ ]:
x_1 = -15 # Minimum x-axis to plot
x_2 = 10 # Maximum x-axis to plot

colors = ['red','olive','navy']

# Parameters for uniform distribution
u_min = [0., 2., 3.]
delta_u = [1., 3., 6.]

# Parameters for gamma distribution
gamma_scale = [0.5, 1, 1.5]
gamma_shape = [0.5, 1, 2]

# Parameters for expon distribution
expon_scale = [0.5, 1.5, 3]

# Parameters for Rayleigh distribution
rayleigh_scale = [0.5, 2, 4]

# Parameters for Weibull distribution
weibull_scale = [1, 2, 4]
weibull_shape = [0.5, 2, 10]

# Parameters for Pareto distribution
pareto_shape = [3, 1, 0.5]

# Parameters for half-norm distribution
halfnorm_scale = [0.5, 2, 4]

# Parameters for lognormal distribution
lognorm_scale = [0.5, 1, 3]
lognorm_shape = [3, 1, 0.5]


fig, axs = plt.subplots(4, 8, figsize=(24, 10))

for i in range(len(u_min)):
    # Define the uniform distribution pdf
    uniform_pdf = uniform.pdf(x=-x, loc=u_min[i], scale=delta_u[i])
    uniform_pdf = uniform_pdf/np.sum(uniform_pdf) # Normalize so area under the curve equals 1
    
    # Convolve the Gaussian and uniform distributions
    uniform_conv = convolve(uniform_pdf, norm_pdf, mode='same')
    uniform_conv = uniform_conv/np.sum(uniform_conv) # Normalize so area under the curve equals 1
    
    axs[0,0].plot(x, uniform_pdf, color=colors[i], label='u (min): '+str(u_min[i])+', delta_u: '+str(delta_u[i]))
    axs[0,0].set_ylim(0,)
    axs[0,0].set_xlim(-10, 0)
    axs[0,0].set_title('uniform distribution')
    axs[0,0].legend(loc='upper left')
    axs[0,0].set_ylabel('Relative probability')
    
    axs[1,0].plot(x, np.cumsum(uniform_pdf), color=colors[i])
    axs[1,0].set_ylim(0,1)
    axs[1,0].set_xlim(-10, 0)
    axs[1,0].set_ylabel('Cumulative probability')

    axs[2,0].plot(x, norm_pdf, '--', color='black', label='Original')
    axs[2,0].plot(x, uniform_conv, '-', color=colors[i], label='Pb-loss')
    axs[2,0].set_xlim(x_1, x_2)
    axs[2,0].set_ylim(0,)
    axs[2,0].set_ylabel('Relative probability')
    
    axs[3,0].plot(x, np.cumsum(norm_pdf), '--', color='black', label='Original')
    axs[3,0].plot(x, np.cumsum(uniform_conv), '-', color=colors[i], label='Pb-loss')
    axs[3,0].set_xlim(x_1, x_2)
    axs[3,0].set_ylim(0,1)
    axs[3,0].set_xlabel('Age offset (%)')
    axs[3,0].set_ylabel('Cumulative probability')

for i in range(len(gamma_scale)):
    # Define the gamma distribution pdf
    gamma_pdf = gamma.pdf(x=-x, loc=0, a=gamma_shape[i], scale=gamma_scale[i])
    gamma_pdf = gamma_pdf/np.sum(gamma_pdf) # Normalize so area under the curve equals 1
    
    # Convolve the Gaussian and gamma distributions
    gamma_conv = convolve(gamma_pdf, norm_pdf, mode='same')
    gamma_conv = gamma_conv/np.sum(gamma_conv) # Normalize so area under the curve equals 1
    
    axs[0,1].plot(x, gamma_pdf, color=colors[i], label='scale: '+str(gamma_scale[i])+', shape: '+str(gamma_shape[i]))
    axs[0,1].set_ylim(0,0.02)
    axs[0,1].set_xlim(-10, 0)
    axs[0,1].set_title('gamma distribution')
    axs[0,1].legend(loc='upper left')
    axs[0,1].set_ylabel('Relative probability')
    
    axs[1,1].plot(x, np.cumsum(gamma_pdf), color=colors[i])
    axs[1,1].set_ylim(0,1)
    axs[1,1].set_xlim(-10, 0)
    axs[1,1].set_ylabel('Cumulative probability')

    axs[2,1].plot(x, norm_pdf, '--', color='black', label='Original')
    axs[2,1].plot(x, gamma_conv, '-', color=colors[i], label='Pb-loss')
    axs[2,1].set_xlim(x_1, x_2)
    axs[2,1].set_ylim(0,)
    axs[2,1].set_ylabel('Relative probability')
    
    axs[3,1].plot(x, np.cumsum(norm_pdf), '--', color='black', label='Original')
    axs[3,1].plot(x, np.cumsum(gamma_conv), '-', color=colors[i], label='Pb-loss')
    axs[3,1].set_xlim(x_1, x_2)
    axs[3,1].set_ylim(0,1)
    axs[3,1].set_xlabel('Age offset (%)')
    axs[3,1].set_ylabel('Cumulative probability')
    
for i in range(len(expon_scale)):
    # Define the expon distribution pdf
    expon_pdf = expon.pdf(x=-x, loc=0, scale=expon_scale[i])
    expon_pdf = expon_pdf/np.sum(expon_pdf) # Normalize so area under the curve equals 1
    
    # Convolve the Gaussian and expon distributions
    expon_conv = convolve(expon_pdf, norm_pdf, mode='same')
    expon_conv = expon_conv/np.sum(expon_conv) # Normalize so area under the curve equals 1
    
    axs[0,2].plot(x, expon_pdf, color=colors[i], label='scale: '+str(expon_scale[i]))
    axs[0,2].set_ylim(0,)
    axs[0,2].set_xlim(-10, 0)
    axs[0,2].set_title('expon distribution')
    axs[0,2].legend(loc='upper left')
    axs[0,2].set_ylabel('Relative probability')
    
    axs[1,2].plot(x, np.cumsum(expon_pdf), color=colors[i])
    axs[1,2].set_ylim(0,1)
    axs[1,2].set_xlim(-10, 0)
    axs[1,2].set_ylabel('Cumulative probability')

    axs[2,2].plot(x, norm_pdf, '--', color='black', label='Original')
    axs[2,2].plot(x, expon_conv, '-', color=colors[i], label='Pb-loss')
    axs[2,2].set_xlim(x_1, x_2)
    axs[2,2].set_ylim(0,)
    axs[2,2].set_ylabel('Relative probability')
    
    axs[3,2].plot(x, np.cumsum(norm_pdf), '--', color='black', label='Original')
    axs[3,2].plot(x, np.cumsum(expon_conv), '-', color=colors[i], label='Pb-loss')
    axs[3,2].set_xlim(x_1, x_2)
    axs[3,2].set_ylim(0,1)
    axs[3,2].set_xlabel('Age offset (%)')
    axs[3,2].set_ylabel('Cumulative probability')
    
for i in range(len(rayleigh_scale)):
    # Define the rayleigh distribution pdf
    rayleigh_pdf = rayleigh.pdf(x=-x, loc=0, scale=rayleigh_scale[i])
    rayleigh_pdf = rayleigh_pdf/np.sum(rayleigh_pdf) # Normalize so area under the curve equals 1
    
    # Convolve the Gaussian and rayleigh distributions
    rayleigh_conv = convolve(rayleigh_pdf, norm_pdf, mode='same')
    rayleigh_conv = rayleigh_conv/np.sum(rayleigh_conv) # Normalize so area under the curve equals 1
    
    axs[0,3].plot(x, rayleigh_pdf, color=colors[i], label='scale: '+str(rayleigh_scale[i]))
    axs[0,3].set_ylim(0,)
    axs[0,3].set_xlim(-10, 0)
    axs[0,3].set_title('Rayleigh distribution')
    axs[0,3].legend(loc='upper left')
    
    axs[1,3].plot(x, np.cumsum(rayleigh_pdf), color=colors[i])
    axs[1,3].set_ylim(0,1)
    axs[1,3].set_xlim(-10, 0)

    axs[2,3].plot(x, norm_pdf, '--', color='black', label='Original')
    axs[2,3].plot(x, rayleigh_conv, '-', color=colors[i], label='Pb-loss')
    axs[2,3].set_xlim(x_1, x_2)
    axs[2,3].set_ylim(0,)
    
    axs[3,3].plot(x, np.cumsum(norm_pdf), '--', color='black', label='Original')
    axs[3,3].plot(x, np.cumsum(rayleigh_conv), '-', color=colors[i], label='Pb-loss')
    axs[3,3].set_xlim(x_1, x_2)
    axs[3,3].set_ylim(0,1)
    axs[3,3].set_xlabel('Age offset (%)')
    
for i in range(len(weibull_scale)):
    # Define the weibull distribution pdf
    weibull_pdf = weibull_min.pdf(x=-x, loc=0, scale=weibull_scale[i], c=weibull_shape[i])
    weibull_pdf = weibull_pdf/np.sum(weibull_pdf) # Normalize so area under the curve equals 1
    
    # Convolve the Gaussian and weibull distributions
    weibull_conv = convolve(weibull_pdf, norm_pdf, mode='same')
    weibull_conv = weibull_conv/np.sum(weibull_conv) # Normalize so area under the curve equals 1
    
    axs[0,4].plot(x, weibull_pdf, color=colors[i], label='scale: '+str(weibull_scale[i])+', shape: '+str(weibull_shape[i]))
    axs[0,4].set_ylim(0,)
    axs[0,4].set_xlim(-10, 0)
    axs[0,4].set_title('Weibull distribution')
    axs[0,4].legend(loc='upper left')
    axs[0,4].set_ylabel('Relative probability')
    
    axs[1,4].plot(x, np.cumsum(weibull_pdf), color=colors[i])
    axs[1,4].set_ylim(0,1)
    axs[1,4].set_xlim(-10, 0)
    axs[1,4].set_ylabel('Cumulative probability')

    axs[2,4].plot(x, norm_pdf, '--', color='black', label='Original')
    axs[2,4].plot(x, weibull_conv, '-', color=colors[i], label='Pb-loss')
    axs[2,4].set_xlim(x_1, x_2)
    axs[2,4].set_ylim(0,)
    axs[2,4].set_ylabel('Relative probability')
    
    axs[3,4].plot(x, np.cumsum(norm_pdf), '--', color='black', label='Original')
    axs[3,4].plot(x, np.cumsum(weibull_conv), '-', color=colors[i], label='Pb-loss')
    axs[3,4].set_xlim(x_1, x_2)
    axs[3,4].set_ylim(0,1)
    axs[3,4].set_xlabel('Age offset (%)')
    axs[3,4].set_ylabel('Cumulative probability')
    
for i in range(len(pareto_shape)):
    # Define the pareto distribution pdf
    pareto_pdf = pareto.pdf(x=-x, loc=-1, scale=1, b=pareto_shape[i])
    pareto_pdf = pareto_pdf/np.sum(pareto_pdf) # Normalize so area under the curve equals 1
    
    # Convolve the Gaussian and pareto distributions
    pareto_conv = convolve(pareto_pdf, norm_pdf, mode='same')
    pareto_conv = pareto_conv/np.sum(pareto_conv) # Normalize so area under the curve equals 1
    
    axs[0,5].plot(x, pareto_pdf, color=colors[i], label='shape: '+str(pareto_shape[i]))
    axs[0,5].set_ylim(0,0.02)
    axs[0,5].set_xlim(-10, 0)
    axs[0,5].set_title('Pareto distribution')
    axs[0,5].legend(loc='upper left')
    
    axs[1,5].plot(x, np.cumsum(pareto_pdf), color=colors[i])
    axs[1,5].set_ylim(0,1)
    axs[1,5].set_xlim(-10, 0)

    axs[2,5].plot(x, norm_pdf, '--', color='black', label='Original')
    axs[2,5].plot(x, pareto_conv, '-', color=colors[i], label='Pb-loss')
    axs[2,5].set_xlim(x_1, x_2)
    axs[2,5].set_ylim(0,)
    
    axs[3,5].plot(x, np.cumsum(norm_pdf), '--', color='black', label='Original')
    axs[3,5].plot(x, np.cumsum(pareto_conv), '-', color=colors[i], label='Pb-loss')
    axs[3,5].set_xlim(x_1, x_2)
    axs[3,5].set_ylim(0,1)
    axs[3,5].set_xlabel('Age offset (%)')
    
for i in range(len(halfnorm_scale)):
    # Define the halfnorm distribution pdf
    halfnorm_pdf = halfnorm.pdf(x=-x, loc=0, scale=halfnorm_scale[i])
    halfnorm_pdf = halfnorm_pdf/np.sum(halfnorm_pdf) # Normalize so area under the curve equals 1
    
    # Convolve the Gaussian and halfnorm distributions
    halfnorm_conv = convolve(halfnorm_pdf, norm_pdf, mode='same')
    halfnorm_conv = halfnorm_conv/np.sum(halfnorm_conv) # Normalize so area under the curve equals 1
    
    axs[0,6].plot(x, halfnorm_pdf, color=colors[i], label='scale: '+str(halfnorm_scale[i]))
    axs[0,6].set_ylim(0,)
    axs[0,6].set_xlim(-10, 0)
    axs[0,6].set_title('halfnorm distribution')
    axs[0,6].legend(loc='upper left')
    
    axs[1,6].plot(x, np.cumsum(halfnorm_pdf), color=colors[i])
    axs[1,6].set_ylim(0,1)
    axs[1,6].set_xlim(-10, 0)

    axs[2,6].plot(x, norm_pdf, '--', color='black', label='Original')
    axs[2,6].plot(x, halfnorm_conv, '-', color=colors[i], label='Pb-loss')
    axs[2,6].set_xlim(x_1, x_2)
    axs[2,6].set_ylim(0,)
    
    axs[3,6].plot(x, np.cumsum(norm_pdf), '--', color='black', label='Original')
    axs[3,6].plot(x, np.cumsum(halfnorm_conv), '-', color=colors[i], label='Pb-loss')
    axs[3,6].set_xlim(x_1, x_2)
    axs[3,6].set_ylim(0,1)
    axs[3,6].set_xlabel('Age offset (%)')
    
for i in range(len(lognorm_shape)):
    # Define the lognorm distribution pdf
    #lognorm_pdf = lognorm.pdf(-x/u)
    lognorm_pdf = lognorm.pdf(x=-x, loc=0, s=lognorm_shape[i], scale=lognorm_scale[i])
    lognorm_pdf = lognorm_pdf/np.sum(lognorm_pdf) # Normalize so area under the curve equals 1
    
    # Convolve the Gaussian and lognorm distributions
    lognorm_conv = convolve(lognorm_pdf, norm_pdf, mode='same')
    lognorm_conv = lognorm_conv/np.sum(lognorm_conv) # Normalize so area under the curve equals 1
    
    axs[0,7].plot(x, lognorm_pdf, color=colors[i], label='scale: '+str(lognorm_scale[i])+', shape: '+str(lognorm_shape[i]))
    axs[0,7].set_ylim(0,0.02)
    axs[0,7].set_xlim(-10, 0)
    axs[0,7].set_title('lognorm distribution')
    axs[0,7].legend(loc='upper left')
    
    axs[1,7].plot(x, np.cumsum(lognorm_pdf), color=colors[i])
    axs[1,7].set_ylim(0,1)
    axs[1,7].set_xlim(-10, 0)

    axs[2,7].plot(x, norm_pdf, '--', color='black', label='Original')
    axs[2,7].plot(x, lognorm_conv, '-', color=colors[i], label='Pb-loss')
    axs[2,7].set_xlim(x_1, x_2)
    axs[2,7].set_ylim(0,)
    
    axs[3,7].plot(x, np.cumsum(norm_pdf), '--', color='black', label='Original')
    axs[3,7].plot(x, np.cumsum(lognorm_conv), '-', color=colors[i], label='Pb-loss')
    axs[3,7].set_xlim(x_1, x_2)
    axs[3,7].set_ylim(0,1)
    axs[3,7].set_xlabel('Age offset (%)')
    
plt.tight_layout()